In [2]:
import pandas as pd
import numpy as np

In [3]:
rules = pd.read_csv("./dataset.csv")
dataset = pd.read_csv("./datasetharga.csv")

In [4]:
variabelDataset = dataset.columns.tolist()
# variabelDataset.remove("harga jual")
variabelDataset

['tahun motor', 'harga beli', 'kondisi', 'harga jual']

In [5]:
maxValueVariable = dataset.max().to_frame().T
minValueVariable = dataset.min().to_frame().T
# mean = (dataset.min() + (dataset.max() - dataset.min()) / 2).to_frame().T
# mean

In [6]:
meanValueVariable = pd.DataFrame(
    {
        "tahun motor": [2014],
        "harga beli": [12225000],
        "kondisi": [89.5],
        "harga jual": [13275000],
    }
)
meanValueVariable

,tahun motor,harga beli,kondisi,harga jual
0,2014,12225000,89.5,13275000


In [7]:
def linearNaik(x, a, b):
    print(" naik")
    if x <= a:
        return 0
    elif a < x < b:
        return (x - a) / (b - a)
    else:
        return 1

In [8]:
def linearTurun(x, a, b):
    print(" turun")
    if x <= a:
        return 1
    elif a <= x <= b:
        return (b - x) / (b - a)
    else:
        return 0

In [9]:
def linearSegitiga(x, a, b, label):
    mid = meanValueVariable[label].iloc[0]
    if a <= x <= mid:
        return linearNaik(x, a, mid)
    elif b >= x >= mid:
        return linearTurun(x, mid, b)
    else:
        return 0

In [10]:
def agregasi_naik(alfa, a, b):
    print(" agre naik")
    return alfa * (b - a) + a

In [11]:
def agregasi_turun(alfa, a, b):
    print(" agre turun")
    return b - (alfa * (b - a))

In [12]:
# sample_data = dataset.sample(n=1)
# sample_rules = rules.sample(n=1)
sample_data = dataset.iloc[0].to_frame().T
sample_rules = rules.iloc[0:27]
sample_rules = sample_rules.drop(columns=["aturan"])

In [13]:
def kategoriTurun(par):
    return par == "Lama" or par == "Murah" or par == "Jelek"

In [14]:
def kategoriSegiTiga(par):
    return (
        par == "Agak Lama" or par == "Normal" or par == "Sedang" or par == "Agak Mahal"
    )

In [15]:
def kategoriNaik(par):
    return par == "Baru" or par == "Mahal" or par == "Bagus"

In [16]:
def isHargaJual(par):
    return par == "harga jual"

In [17]:
def defuzifikasi(r, z):
    lenArr = len(r)
    if len(r) != len(z):
        return

    result = 0
    for i in range(lenArr):
        product = r[i] * z[i]
        result += product

    return result / sum(r)

In [22]:
result = []
for i in range(sample_data.shape[0]):
    r = []
    z = []
    for x in range(sample_rules.shape[0]):
        varRules = []
        for colName in sample_rules:
            if kategoriTurun(sample_rules[colName].iloc[x]):
                print(sample_rules[colName].iloc[x], end="")
                if isHargaJual(colName):
                    minR = round(min(varRules), 3)
                    agregasi = agregasi_turun(
                        minR,
                        minValueVariable[colName][0],
                        meanValueVariable[colName][0],
                    )
                    z.append(agregasi)
                    r.append(minR)
                    print(agregasi)
                    break
                varRules.append(
                    linearTurun(
                        sample_data[colName].iloc[i],
                        minValueVariable[colName].iloc[0],
                        meanValueVariable[colName].iloc[0],
                    )
                )
            elif kategoriSegiTiga(sample_rules[colName].iloc[x]):
                print(sample_rules[colName].iloc[x], end="")
                if isHargaJual(colName):
                    minR = round(min(varRules), 3)
                    if sample_data[colName].iloc[i] > meanValueVariable[colName][0]:
                        agregasi = agregasi_turun(
                            minR,
                            meanValueVariable[colName][0],
                            maxValueVariable[colName][0],
                        )
                    elif sample_data[colName].iloc[i] < meanValueVariable[colName][0]:
                        agregasi = agregasi_naik(
                            minR,
                            minValueVariable[colName][0],
                            meanValueVariable[colName][0],
                        )
                    else:
                        agregasi = meanValueVariable[colName][0]
                    z.append(agregasi)
                    r.append(minR)
                    break
                varRules.append(
                    linearSegitiga(
                        sample_data[colName].iloc[i],
                        minValueVariable[colName].iloc[0],
                        maxValueVariable[colName].iloc[0],
                        colName,
                    )
                )
            else:
                print(sample_rules[colName].iloc[x], end="")
                if isHargaJual(colName):
                    minR = round(min(varRules), 3)
                    agregasi = agregasi_naik(
                        minR,
                        meanValueVariable[colName][0],
                        maxValueVariable[colName][0],
                    )
                    r.append(minR)
                    z.append(agregasi)
                    break
                varRules.append(
                    linearNaik(
                        sample_data[colName].iloc[i],
                        meanValueVariable[colName].iloc[0],
                        maxValueVariable[colName].iloc[0],
                    )
                )
            print(varRules)
        print(f"nilai min rules baris ke-{x+1} = {r[x]} ", end="")
        print(f"dengan nilai z {z[x]}")

    print(f"hasil prediksi {round(defuzifikasi(r, z), 2)}")

Lama turun
[0]
Murah turun
[0, 0]
Jelek turun
[0, 0, 0]
Murah agre turun
13275000
nilai min rules baris ke-1 = 0 dengan nilai z 13275000
Lama turun
[0]
Murah turun
[0, 0]
Sedang turun
[0, 0, 0.9473684210526315]
Murah agre turun
13275000
nilai min rules baris ke-2 = 0 dengan nilai z 13275000
Lama turun
[0]
Murah turun
[0, 0]
Bagus naik
[0, 0, 0.05263157894736842]
Murah agre turun
13275000
nilai min rules baris ke-3 = 0 dengan nilai z 13275000
Lama turun
[0]
Normal turun
[0, 0.7437185929648241]
Jelek turun
[0, 0.7437185929648241, 0]
Agak Mahal agre turun
nilai min rules baris ke-4 = 0 dengan nilai z 18250000
Lama turun
[0]
Normal turun
[0, 0.7437185929648241]
Sedang turun
[0, 0.7437185929648241, 0.9473684210526315]
Agak Mahal agre turun
nilai min rules baris ke-5 = 0 dengan nilai z 18250000
Lama turun
[0]
Normal turun
[0, 0.7437185929648241]
Bagus naik
[0, 0.7437185929648241, 0.05263157894736842]
Agak Mahal agre turun
nilai min rules baris ke-6 = 0 dengan nilai z 18250000
Lama turun
[0]


In [19]:
agregasi_turun(
    0.053,
    meanValueVariable["harga jual"][0],
    maxValueVariable["harga jual"][0],
)

 agre turun


17986325.0

In [20]:
a = [0, 0, 0, 0, 0]
b = [13275000, 13275000, 13275000, 18250000, 18250000]

In [21]:
division by zero

SyntaxError: invalid syntax (<ipython-input-21-e0a270eac36b>, line 1)